# Edit Annotations

Thanks for using this jupyter notebook. This notebook covers several useful functions to play around with the annotations we have. The list below summarises where you can find codes for each function.

### **[Almost always needed]** Importing: 
Import JSON into DataFrame - Function (1)

### Conversions:
* Convert output from Detectron2 to human readable format (also for editing in CVAT) - Function (1a)
* Convert from COCO json to YOLOv7 txt input - Function (3)
* Convert from YOLOv7 txt output to COCO json - Function (4)
* Convert two jsons into one - Function (5)

### Queries and counting: *[in anntoations.ipynb]*

~~* Count the number of instances of a certain category in the json - Function (2a)~~

~~* Count the category (and/or clicks) distribution - Function (2)~~

~~* Move wanted images indicated in a json from a library of images - Function (2b)~~

### Special tasks: *[in annotations.ipynb]*

~~* Trim a selection of annotation from a master json - Function (6)~~

~~* create json for the split subset (for training, validation etc..)~~

~~* create empty json for the split subset (let Detectron do its job)~~

~~* create json for the remainders of the split~~

~~* Clean up the json of the validation set - Function (7)~~

# 1) Import JSON into DataFrame

Export annotations from CVAT as COCO JSON. 

Put the json file in the working directory and change the variable "filename".

In [6]:
import pandas as pd
import numpy as np
import json

# utilities

def findCategory(data):
    # find categories
    cats = data["categories"]
    category = pd.DataFrame(cats)
    category = category.drop(['supercategory'], axis=1)
    category = category.rename(columns={'id': 'category_id'})
    return category

def findImages(data):
    img = data["images"]
    images = pd.DataFrame(img)
    
    # unwanted columns exist if exported from CVAT. Not if generated by my code
    if set(['license','flickr_url','coco_url','date_captured']).issubset(images.columns):
        images = images.drop(columns=['license','flickr_url','coco_url','date_captured'])
    
    return images

def findAnnotations(data):
    anno = data["annotations"]
    df = pd.DataFrame(anno)
    return df

def cleanForJson(category=None, df=None):
    # clean category for json dump
    if category is not None:
        category = category.rename(columns={'category_id': 'id'})
        category['supercategory'] = ""

    # add columns in df for json dump
    if df is not None:
        df['iscrowd'] = 0
        df['attributes'] = [{'occluded':False}] * len(df['id'])
        cols = ['id', 'image_id', 'category_id', 'segmentation', 'area', 'bbox', 'iscrowd', 'attributes']
        df = df[cols + [c for c in df.columns if c not in cols]]
    
    return category, df

def sort_reid(dataframe, target_column, reindex_column):
    df2 = dataframe.sort_values(by=[target_column], ascending=True)     # create a new df with the target column sorted in ascending order
    df2 = df2.reset_index(drop=True)                                    # we don't need the added column of old index
    df2[reindex_column] = df2.index + 1
    return df2

# convert all np.integer, np.floating and np.ndarray into json recognisable int, float and lists
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [7]:
import pandas as pd
import numpy as np
import re
import json

df = pd.DataFrame()

# filename = './input/train.json'
filename = './input/a14-p_val.json'
# filename = './input/Corr/val_lim4.json'

with open(filename, 'r') as file:
    data = json.load(file)
    
    if "categories" in data:
        category = findCategory(data)
    
    if "images" in data:
        images = findImages(data)
        nos_image = images['id'].nunique()

    df = findAnnotations(data)


## 1a) Fix annotations inferred from Detectron2

Coco_instances_results.json from COCOEvaluator only contains annotations. We need to decode the masks which are encoded in RLE for memory efficiency.

Besides, we also supplement the categories from train.json and the list of images from the test/val folder (***remember to change the directory***).

The final window outputs a json that combines the categories, the list of images and annotations. Feel free to use it if converting outputs from Detectron2 to CVAT-readable outputs is the main purpose.

In [4]:
import cv2
import pycocotools.mask as mask_util

# Convert RLE counts into segmentation masks
def polygonFromMask(maskedArr): # https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py

    contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    segmentation = []
    for contour in contours:
        # Valid polygons have >= 6 coordinates (3 points)
        if contour.size >= 6:
            segmentation.append(contour.flatten().tolist())
    RLEs = mask_util.frPyObjects(segmentation, maskedArr.shape[0], maskedArr.shape[1])
    RLE = mask_util.merge(RLEs)
    # RLE = mask.encode(np.asfortranarray(maskedArr))
    area = mask_util.area(RLE)
    [x, y, w, h] = cv2.boundingRect(maskedArr)

    return segmentation[0], area #, [x, y, w, h], area

In [3]:
import pandas as pd
import numpy as np
import re
import json

# df = pd.DataFrame()
filename = './input/coco_instances_results.json'

with open(filename, 'r') as file:
    data = json.load(file)
    df = pd.DataFrame(data)

# add annotation id, iscrowd and attributes
df['id'] = df.index + 1
df['iscrowd'] = 0
df.loc[:, 'attributes'] = [{"occluded":False}] * len(df['id'])
df.insert(1, "area", "")

# decode Detectron masks in RLE format
for i in range(len(df['segmentation'])):
    sample_mask = mask_util.decode(df['segmentation'][i])
    sample_polygon = polygonFromMask(sample_mask)
    seg_concat = [[]]
    seg_concat[0][:] = sample_polygon[0]
    df['segmentation'][i] = seg_concat
    # Find area
    # coord = np.transpose(np.array([seg_concat[0][::2],seg_concat[0][1::2]])) # Polygon work with dimensions (n, 2)
    df['area'][i] = sample_polygon[1]

# sort bbox to 2 decimal places
for i in range(len(df['bbox'])):
    df['bbox'][i] = [round(j,2) for j in df['bbox'][i]]

columnsTitles = ['id', 'image_id', 'category_id', 'segmentation', 'area', 'bbox', 'iscrowd', 'attributes', 'score']
df = df.reindex(columns=columnsTitles)

# df

/tmp/ipykernel_2903/1252471046.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmentation'][i] = seg_concat
/tmp/ipykernel_2903/1252471046.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['area'][i] = sample_polygon[1]
/tmp/ipykernel_2903/1252471046.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bbox'][i] = [round(j,2) for j in df['bbox'][i]]


In [5]:
import glob, os
from PIL import Image

CAT_PATH = '../images/val/val.json'
IMAGE_CP_PATH = '../images/test/SS4/SS4_test.json'

with open(CAT_PATH, 'r') as file:
    data = json.load(file)
    category = findCategory(data)
# clean category for json dump
category = category.rename(columns={'category_id': 'id'})
category['supercategory'] = ""

# copy image metadata from val.json. This preserves the image_id used when inferring pseudolabels for roads_val, which uses metadata from ./val.json
with open(IMAGE_CP_PATH, 'r') as file2:
    data2 = json.load(file2)
    images = findImages(data2)

# or you can create the DataFrame "images" from scratch...
cp_new = False
if cp_new == True:
    IMAGE_DIR = '../images/val/'
    IMAGE_PATHS = glob.glob(os.path.join(IMAGE_DIR, '*.jp*g'))

    for image_id, image_path in enumerate(IMAGE_PATHS):
        img = Image.open(image_path)
        if image_id == 0:
            images = pd.DataFrame(columns=['id','width','height','file_name'])

        add = dict()
        add['id'] = image_id + 1
        add['width'], add['height'] = img.size
        add['file_name']= os.path.basename(img.filename)
        # add['file_name'] = re.findall(r'^.*\\([^\/]*)$',img.filename)[-1]
        add_im = pd.DataFrame(add, index=[0])
        images = pd.concat([images, add_im], ignore_index = True)   

nos_image = images['id'].nunique()

In [6]:
# dump a combined json

# Write categories, images and annotations as arrays containing individual entries as a dictionary
# see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html for to_dict styles
dict_to_json = {
    "categories": category.to_dict('records'),
    "images": images.to_dict('records'),
    "annotations": df.to_dict('records')
    }

with open("./SS4_inferred.json", "w") as outfile:
    json.dump(dict_to_json, outfile, cls=NpEncoder)

# 3) Convert COCO JSON to YOLO Labels File

In [8]:
import re, os
import yaml

# For training without specific categories
cat_elim = [1, 2, 5, 6, 7, 8, 9, 11, 12, 13]
out_dir = './labels'
rearr_cat = True

# Create the df of categories I want
df_want = df[~df['category_id'].isin(cat_elim)]
category_want = category[~category['category_id'].isin(cat_elim)]
# rearrange the category_id to start from 1
if rearr_cat == True:
    category_want.rename(columns={'category_id': 'category_id_old'}, inplace=True)
    category_want = sort_reid(category_want, 'category_id_old', 'category_id')
    df_want = df_want.merge(category_want[['category_id','category_id_old']], left_on='category_id', right_on='category_id_old')
    df_want = df_want.drop(columns=['category_id_old','category_id_x'])
    df_want = df_want.rename(columns={'category_id_y': 'category_id'})
    category_want = category_want.drop(columns=['category_id_old'])
# images_want = images[images['id'].isin(df_want['image_id'])]            # delete in images the images that are not in df_want
images = images
category = category_want
df = sort_reid(df_want, 'image_id', 'id')
del df_want, category_want

# find if image_id has any defects
os.makedirs(out_dir, exist_ok=True)
for i in images['id']:
    if (i in df['image_id'].unique()) == True:
        # print(f'image {i} has defect')
        
        # Concatenate df and images for the text file
        df2 = df[df['image_id']==i]
        image2 = images.loc[(images['id']==i),['id','width','height','file_name']]
        image2 = image2.rename(columns={'id': 'image_id'})
        df2 = df2.drop(columns=['area','bbox','iscrowd','attributes'])
        df2 = df2.merge(image2,left_on='image_id',right_on='image_id') 
        
        # Normalise polygon points
        poly_print = []
        im_width = df2['width'][0]
        im_height = df2['height'][0]
        for f, x in enumerate(df2['segmentation'].to_list()):
            # if df2['category_id'][f] in cat_elim:
            #     # skip unwanted categories
            #     continue
            # else:
            poly = [df2['category_id'][f]-1]
            # print(f'image {i} polygon length {len(x[0])}')
            for y in range(1,len(x[0]),2):
                # print(x,y)
                ptx = x[0][y-1]/im_width
                pty = x[0][y]/im_height
                poly.extend([round(min(ptx,1),4), round(min(pty, 1),4)])              # set mins to 1 to prevent any out-of-bounds conversions
            # poly.extend([poly[1],poly[2]])                      # YOLO does not need polygon points to form a close loop
            poly_print.append(poly)
            poly = []
        
        # fname = re.findall(r'(.*)(?:.jpg)$',df2['file_name'][0])[0]
        fname = os.path.splitext(df2['file_name'][0])[0]
        fname = os.path.join(out_dir, f'{fname}.txt')
        with open(fname, "w") as outfile:
            for num, j in enumerate(poly_print):
                outfile.write(' '.join(str(e) for e in j))
                # print(f'image {i} defect {num} of {len(poly_print)}')
                if num==len(poly_print)-1: outfile.write('')
                else: outfile.write('\n')
        poly_print = []
    else:
        # fname = re.findall(r'(.*)(?:.jpg)$',images.loc[(images['id']==i),'file_name'].values[0])[0]
        fname = os.path.splitext(images.loc[(images['id']==i),'file_name'].values[0])[0]
        fname = os.path.join(out_dir, f'{fname}.txt')
        with open(fname, "w") as outfile:
            outfile.write('')

print(f'Labels of {nos_image} images prepared!')

# Create data.yaml file
nc = len(category['category_id'])
names = category['name'].to_list()
with open(r'./custom.yaml', "w") as outfile:
    outfile.write('train: a14-p/train/images \n')
    outfile.write('val: a14-p/val/images \n')
    outfile.write('test: a14-p/test/images \n \n')
    outfile.write(f'nc: {nc} \n')
    outfile.write(f'names: {names} \n')
print('custom.yaml prepared!')

# np.array(df2['segmentation'].to_list()[0])/10

/tmp/ipykernel_10317/1271686149.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_want.rename(columns={'category_id': 'category_id_old'}, inplace=True)


Labels of 557 images prepared!
custom.yaml prepared!


## Instructions

Now move the folder "txt_labels" to the corresponding ./data/[test/train/val] folder

Move also a the custom.yaml to ./data

Leave the folder "txt_labels" in this ./fix_annotation folder for subsequent annotation files.

Happy training!

# 4) Convert YOLO Labels to COCO JSON

In [2]:
import glob, os
import yaml
import pandas as pd
import numpy as np
import re
import json
from PIL import Image
# from imantics import Mask
from shapely.geometry import Polygon

IMAGE_DIR = '../data/test/images/'
# IMAGE_DIR = os.path.join(os.getcwd(), '..' ) + '/data/test/images/'
IMAGE_PATHS = glob.glob(os.path.join(IMAGE_DIR, '*.jp*g'))
LABEL_DIR = './input/'
LABEL_PATHS = glob.glob(os.path.join(LABEL_DIR, '*.txt'))
# SAVE_DIR = './labels/test_annotated/'

with open("custom.yaml", "r") as cat:
    data = yaml.safe_load(cat)
    category = pd.DataFrame(data['names'], columns=['name'])
    category['id'] = category.index + 1
    category['supercategory'] = ""
    category = category[['id','name','supercategory']]


for image_id, image_path in enumerate(IMAGE_PATHS):
    img = Image.open(image_path)
    if image_id == 0:
        image = pd.DataFrame(columns=['id','width','height','file_name'])

    add = dict()
    add['id'] = image_id + 1
    add['width'], add['height'] = img.size
    # add['file_name']= img.filename.split("/")[-1]
    add['file_name'] = re.findall(r'^.*\\([^\/]*)$',img.filename)[-1]
    add_im = pd.DataFrame(add, index=[0])
    image = pd.concat([image, add_im], ignore_index = True)   



for txt_id, txt_path in enumerate(LABEL_PATHS):
    if txt_id == 0:
        df = pd.DataFrame(columns=['id','image_id','category_id','segmentation','area','bbox','iscrowd', 'attributes'])
        full_anno = pd.DataFrame()
    
    with open(txt_path, 'r', encoding='utf-8-sig', errors="surrogateescape") as file:
        # if df.shape[0]==0:
        if len(df['id']) == 0:
            prev_defect = 0
        else:
            prev_defect = max(df['id'])
        
        for i, line in enumerate(file):
            inner_list = [float(elt.strip()) for elt in line.split(' ')]
            defect = dict()
            
            filename = re.findall(r'^.*\\([^\/]*)(?:s|_mask.txt)$',txt_path)[-1] + ".jpg"
            # filename = re.findall(r'^.*\\([^\/]*)(?:s|.txt)$',txt_path)[-1] + ".jpg"
            defect['id'] = prev_defect + i + 1
            defect['image_id'] = image.loc[(image['file_name']==filename), 'id'].values[0]
            # defect['image_id'] = txt_id + 1
            defect['category_id'] = int(inner_list[0]) + 1
            
            # upscale segmentation
            seg = np.array(inner_list[1:])
            x_scale = image.loc[(image['id']==defect['image_id']), 'width'].values      # x width
            y_scale = image.loc[(image['id']==defect['image_id']), 'height'].values     # y height
            # print(x_scale, y_scale)
            for y in range(1,len(seg),2):
                seg[y] = round(float(seg[y] * y_scale),1)
                seg[y-1] = round(float(seg[y-1] * x_scale),1)    
            seg_concat = [[]]
            seg_concat[0][:] = seg.tolist()
            defect['segmentation'] = seg_concat
            # defect['segmentation'] = seg.to_list()

            # Find bbox
            x_min, x_max = min(seg[::2]), max(seg[::2])
            y_min, y_max = min(seg[1::2]), max(seg[1::2])
            defect['bbox'] = [x_min, y_min, round(x_max-x_min,1), round(y_max-y_min,1)]
            # Find area
            coord = np.transpose(np.array([seg[::2],seg[1::2]])) # Polygon work with dimensions (n, 2)
            try:
                defect['area'] = round(Polygon(coord).area,1)
            except:
                print('defect id: {}, image: {}'.format(defect['id'], image.loc[(image['id']==defect['image_id']), 'file_name'].values))
                continue
            defect['iscrowd'] = 0
            defect['attributes'] = {"occluded":False}

            add_df = pd.DataFrame([defect], index=[0])
            df = pd.concat([df, add_df], ignore_index = True)


# Write categories, images and annotations as arrays containing individual entries as a dictionary
# see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html for to_dict styles
dict_to_json = {
    "categories": category.to_dict('records'),
    "images": image.to_dict('records'),
    "annotations": df.to_dict('records')
    }

# convert all np.integer, np.floating and np.ndarray into json recognisable int, float and lists
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open("./inferred.json", "w") as outfile:
    json.dump(dict_to_json, outfile, cls=NpEncoder)

df

defect id: 139, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000020_003239_Lane2.jpg']
defect id: 148, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000020_003239_Lane2.jpg']
defect id: 162, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000020_003242_Lane2.jpg']
defect id: 187, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000020_003357_Lane2.jpg']
defect id: 226, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000023_000238_Lane1.jpg']
defect id: 277, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000023_000345_Lane1.jpg']
defect id: 307, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000023_000392_Lane1.jpg']
defect id: 387, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000023_000671_Lane1.jpg']
defect id: 564, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_

,id,image_id,category_id,segmentation,area,bbox,iscrowd,attributes
0,1,1,11,"[[2410.0, 638.8, 2406.3, 642.5, 2406.3, 653.8,...",3226.7,"[2402.4, 638.8, 57.7, 64.1]",0,{'occluded': False}
1,2,2,11,"[[2329.2, 393.1, 2325.3, 396.8, 2302.4, 396.8,...",17230.7,"[2294.7, 393.1, 165.4, 147.4]",0,{'occluded': False}
2,3,2,11,"[[223.2, 362.9, 219.5, 366.6, 188.7, 366.6, 18...",11394.8,"[142.4, 362.9, 146.4, 98.3]",0,{'occluded': False}
3,4,4,11,"[[950.9, 75.7, 947.2, 79.4, 931.6, 79.4, 927.9...",12392.9,"[924.0, 75.7, 219.5, 64.1]",0,{'occluded': False}
4,5,5,11,"[[308.0, 211.6, 304.1, 215.5, 296.4, 215.5, 28...",17849.3,"[246.4, 211.6, 219.5, 98.2]",0,{'occluded': False}
...,...,...,...,...,...,...,...,...
714,726,850,11,"[[169.5, 302.4, 165.6, 306.1, 138.7, 306.1, 13...",29376.9,"[76.9, 302.4, 396.7, 192.7]",0,{'occluded': False}
715,727,853,11,"[[365.7, 166.3, 362.0, 170.0, 288.8, 170.0, 28...",25299.0,"[215.6, 166.3, 358.0, 162.5]",0,{'occluded': False}
716,728,854,11,"[[111.6, 393.1, 107.7, 396.8, 104.0, 396.8, 10...",57148.7,"[0.0, 393.1, 512.0, 222.9]",0,{'occluded': False}
717,729,855,11,"[[585.2, 151.1, 581.3, 155.0, 362.0, 155.0, 35...",22138.1,"[261.7, 151.1, 389.0, 128.5]",0,{'occluded': False}


# 5) Combine COCO JSON

Useful when patches and crack_longitudinal are detected using one trained model, while other defects are detected with another

In [2]:
# Functions to fiddle around with the json file
import json
import pandas as pd

def createDF(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
        
        category = findCategory(data)
        images = findImages(data)
        nos_image = images['id'].max()
        df = findAnnotations(data)
        df = df.merge(images[['id','file_name']], left_on='image_id', right_on='id')
        df = df.rename(columns={'id_x': 'id'})
        df = drop_columns_if_exist(df,columns=['iscrowd','attributes','id_y'])
        return category, images, df

def drop_columns_if_exist(df, columns):
    df = df.copy()
    for col in columns:
        if col in df.columns:
            df = df.drop(columns=col)
    return df

# if categories aren't supposed to be the same
# category 1 and df are the ones you want to change
def fix_category_id(category1, category2, df, df2):
    
    category_name = pd.concat([category1['name'], category2['name']], ignore_index = True)      # combine category 1 and 2
    unique_category_names = category_name.unique()                                              # Get unique category names
    # Create a new DataFrame with these unique category names and a new column for the relisted category_id
    category_new = pd.DataFrame({
        'category_id': range(1, len(unique_category_names) + 1),
        'name': unique_category_names
    })

    category1 = category1.merge(category_new, on='name', how='left')        # map category_id to category1 and category 2
    category2 = category2.merge(category_new, on='name', how='left')        # map category_id to category1 and category 2

    # use new category_id to replace old category_id in df
    df = df.merge(category1[['category_id_x', 'category_id_y']], left_on='category_id', right_on='category_id_x', how='left')
    df2 = df2.merge(category2[['category_id_x', 'category_id_y']], left_on='category_id', right_on='category_id_x', how='left')
    df = df.drop(columns=['category_id', 'category_id_x']).rename(columns={'category_id_y': 'category_id'})
    df2 = df2.drop(columns=['category_id', 'category_id_x']).rename(columns={'category_id_y': 'category_id'})

    return category_new, df, df2


In [3]:
# Functions to combine segmentation masks
from pycocotools import mask as cocomask
import cv2

def polygon_to_mask(polygons, height, width):
    '''
    Args:
        polygon (list): in [[...]]
        height, width (int): height and width of image
    Returns:
        np.array: a mask of shape (H, W)
    '''
    the_mask = np.zeros((height, width), dtype=np.uint8)

    # tackle the problem of having >1 polygon in an instance
    if len(polygons) > 1:
        for poly in polygons:
            # poly = np.array(poly).reshape((-1, 2))
            rles = cocomask.frPyObjects([poly], height, width)
            a_mask = np.squeeze(cocomask.decode(rles))
            print(a_mask.shape)
            the_mask = np.logical_or(the_mask, a_mask)
    else:
        rles = cocomask.frPyObjects(polygons, height, width)
        the_mask = np.squeeze(cocomask.decode(rles))

    return the_mask

def mask_iou(masks):
    """
    Calculate the IoU of a list of masks.
    Args:
        masks (list): a list of masks of shape (N, H, W).
    Returns:
        np.array: outputs of a matrix of shape (len(masks), len(masks)).
    """
    num_masks = len(masks)
    mask_ious = np.zeros((num_masks, num_masks), dtype=np.float32)

    # Calculate the intersection and union of masks using logical AND and OR operation
    for i in range(num_masks):
        for j in range(i+1, num_masks):
            intersection = np.logical_and(masks[i], masks[j])
            union = np.logical_or(masks[i], masks[j])
            iou = np.sum(intersection) / np.sum(union)
            mask_ious[i, j] = iou
            mask_ious[j, i] = iou

    return mask_ious

def mask_to_polygon(maskedArr):
    """
    Convert a mask in a binary matrix of shape (H, W) to a polygon.
    Args:
        maskedArr (np.array): a binary matrix of shape (H,W) that contains True/False value
    Returns:
        segmentation[0] (np.array): an array of the polygon. [x1, y1, x2, y2...]
        area (float): the size of the polygon
        bbox (list): Bounding box coordinates in COCO format [x_min, y_min, delta_x, delta_y].
    """
    # Convert the binary mask to a binary image (0 and 255 values)
    binary_image = maskedArr.astype(np.uint8) * 255

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Retrieve the polygon coordinates for each contour
    polygons = []
    for contour in contours:
        if contour.size >= 6:
            polygons.append(contour.flatten().tolist())
        # polygon = contour.squeeze().tolist()
        # polygons.append(polygon)
    # Calculate areas and bbox
    area = np.sum(maskedArr)    
    # RLEs = mask_util.frPyObjects(segmentation, maskedArr.shape[0], maskedArr.shape[1])
    # RLE = mask_util.merge(RLEs)
    # area = mask_util.area(RLE)
    bbox = find_remasked_bbox(maskedArr)

    return polygons, area, bbox

def merge_masks(mask1, mask2):
    """
    Merge two masks.
    Args:
        mask1 (np.array): First mask of shape (H, W).
        mask2 (np.array): Second mask of shape (H, W).
    Returns:
        np.array: Merged mask if IoU is above the threshold, otherwise None.
    """
    merged_mask = np.logical_or(mask1, mask2)
    return merged_mask

# also need to fix areas
def find_remasked_bbox(mask):
    """
    Find the bounding box that fits the resized mask.

    Args:
        mask (np.array): Resized mask of shape (H', W').
        original_shape (tuple): Shape of the original image or mask (H, W).

    Returns:
        list: Bounding box coordinates in COCO format [x_min, y_min, delta_x, delta_y].
    """
    # Find the minimum and maximum coordinates that enclose the mask region
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]

    return [x_min, y_min, x_max-x_min, y_max-y_min]


In [4]:
import pandas as pd
import numpy as np
import re
import json

# Files to be combined
filename = '../data/A12AL/A12AL_train4_00.json'
filename2 = '../data/A12AL/A12AL_rest4.json'

# Inputs
iou_merge_thres = 0.5          # IoU for merging polygons
same_category = True          # True if categories are supposed to be the same

# Store categories, images and annotations in separate dataframes
category, images, df = createDF(filename)
category2, images2, df2 = createDF(filename2)

# Categories
if same_category == True:
    # if categories are supposed to be the same 
    # check length
    if len(category) != len(category2):
        print('categories not the same. Check before proceeding')
    # check each category
    for i in range(len(category['name'])):
        if category['name'][i] != category2['name'][i]:
            print('category id: {} , {} in file 1 different from category id: {} , {} in file 2. Please check'.format(category['category_id'][i], category['name'][i], category2['category_id'][i], category2['name'][i]))
    category_new = category
else:
    category_new, df, df2 = fix_category_id(category, category2, df, df2)

category_new, _ = cleanForJson(category=category_new)

# Annotations
# merge two annotation df
df_new = pd.concat([df, df2], ignore_index = True)

# combine and re-arrange the image info
images_new = pd.DataFrame(columns=['id','width','height','file_name'])
images_new['file_name'] = df_new['file_name'].unique()
images_new = images_new.sort_values(by=['file_name'], ignore_index=True)
images_new['id'] = images_new.index + 1
for i in range(len(images_new['file_name'])):
    # find out which json records the concerned image
    if images_new['file_name'][i] in images['file_name'].tolist():
        dim = images.loc[(images['file_name']==images_new['file_name'][i]),['width','height']]
    elif images_new['file_name'][i] in images2['file_name'].tolist():
        dim = images2.loc[(images2['file_name']==images_new['file_name'][i]),['width','height']]
    else:
        print('image not included')
        continue
    # paste width and height info
    images_new.loc[i,'width'] = dim['width'].values[0]
    images_new.loc[i,'height'] = dim['height'].values[0]


# assign new image id and defect id in df_new
for i in range(len(df_new['id'])):
    df_new.loc[i, 'image_id'] = images_new.loc[(images_new['file_name']==df_new['file_name'][i]), 'id'].values
df_new = df_new.sort_values(by=['image_id'], ignore_index=True)
df_new['id'] = df_new.index + 1
_, df_new = cleanForJson(df=df_new)
df_new = df_new.drop(columns=['file_name'])

# combine masks in the same image
for i, v in enumerate(images_new['id']):
    height_i = images_new.loc[i,'height']
    width_i = images_new.loc[i,'width']
    df_check = df_new[df_new['image_id']== v]
    df_check.reset_index(drop=True)
    seg_polygons = df_check['segmentation'].to_list()
    seg_catid = df_check['category_id'].to_list()

    # convert polygons of an image to masks
    seg_masks = []
    for idplygn, x in enumerate(seg_polygons):
        mask = polygon_to_mask(x, height_i, width_i)
        seg_masks.append(mask)
    
    # calculate iou of all instances
    masks_iou = mask_iou(seg_masks)
    
    # merge masks
    for i in range(masks_iou.shape[0]):
        for j in range(i + 1, masks_iou.shape[1]):
            the_mask_iou = masks_iou[i, j]
            if the_mask_iou > iou_merge_thres and seg_catid[i]==seg_catid[j]:
                merged_mask = merge_masks(seg_masks[i],seg_masks[j])
                merged_polygon = mask_to_polygon(merged_mask)
                seg_concat = []
                seg_concat[:] = merged_polygon[0]
                # add the new merged polygon to the i-th entry
                df_check.at[i, 'segmentation'] = seg_concat
                df_check.at[i,'area'] = merged_polygon[1]                     # new area for the new mask
                df_check.at[i,'bbox'] = find_remasked_bbox(merged_mask)       # new bbox for the new mask
                # clear j-th entry 
                df_check.at[j,'segmentation'] = []
                df_check.at[j,'area'] = np.NaN
                df_check.at[j,'bbox'] = []

    df_check = df_check.dropna(subset=['area'])
    df_new = df_new.drop(df_new[df_new.image_id == v].index)
    df_new = pd.concat([df_new,df_check], ignore_index=True)

columnsTitles = ['id', 'image_id', 'category_id', 'segmentation', 'area', 'bbox', 'iscrowd', 'attributes', 'score']
df_new = df_new.reindex(columns=columnsTitles)
df_new['id'] = df_new.index + 1
# clear the "score" column if there are no scores (i.e. combining Ground Truths)
if df_new['score'].isnull().sum() == len(df_new['score']):
    df_new = df_new.drop(columns=['score'])

# Write categories, images and annotations as arrays containing individual entries as a dictionary
# see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html for to_dict styles
dict_to_json = {
    "categories": category_new.to_dict('records'),
    "images": images_new.to_dict('records'),
    "annotations": df_new.to_dict('records')
    }

with open("./A12AL_train4_all.json", "w") as outfile:
    json.dump(dict_to_json, outfile, cls=NpEncoder)


(2056, 2464)
(2056, 2464)


# Dataframe formats for reference

In [10]:
images


,id,width,height,file_name
0,1,1232,1028,A14EBWBJ47AWoolpittoHaugleyBridge_sideview_000...
1,2,1232,1028,A14EBWBJ47AWoolpittoHaugleyBridge_sideview_000...
2,3,1232,1028,A14EBWBJ47AWoolpittoHaugleyBridge_sideview_000...
3,4,1232,1028,A14EBWBJ47AWoolpittoHaugleyBridge_sideview_000...
4,5,1232,1028,A14EBWBJ47AWoolpittoHaugleyBridge_sideview_000...
...,...,...,...,...
5878,5879,1232,1028,A14EBWBJ47AWoolpittoHaugleyBridge_sideview_000...
5879,5880,1232,1028,A14EBWBJ47AWoolpittoHaugleyBridge_sideview_000...
5880,5881,1232,1028,A14EBWBJ47AWoolpittoHaugleyBridge_sideview_000...
5881,5882,1232,1028,A14EBWBJ47AWoolpittoHaugleyBridge_sideview_000...


In [9]:
category

,category_id,name
0,1,bleeding
1,2,raveling
2,3,crack_transverse
3,4,crack_longitudinal
4,5,crack_edge
5,6,crack_alligator
6,7,crack_block
7,8,shoving
8,9,rutting
9,10,potholes


In [8]:
df

,id,image_id,category_id,segmentation,area,bbox,iscrowd,attributes
0,1,10,11,"[[0.0, 425.2, 319.15, 403.27, 682.85, 415.64, ...",160258.0,"[0.0, 341.42, 2464.0, 98.97]",0,{'occluded': False}
1,2,14,4,"[[806.55, 38.64, 854.4, 34.01, 880.64, 64.88, ...",7664.0,"[806.55, 34.01, 422.93, 290.19]",0,{'occluded': False}
2,3,15,11,"[[84.11, 185.55, 145.96, 184.31, 174.42, 145.9...",2652.0,"[84.11, 139.78, 90.31, 45.77]",0,{'occluded': False}
3,4,15,4,"[[947.02, 407.55, 1002.58, 407.55, 1021.11, 44...",8810.0,"[947.02, 407.55, 284.01, 321.05]",0,{'occluded': False}
4,5,15,4,"[[718.57, 6.23, 754.07, 47.9, 794.2, 89.58, 82...",18234.0,"[718.57, 3.14, 327.23, 395.15]",0,{'occluded': False}
...,...,...,...,...,...,...,...,...
5417,5418,5872,12,"[[1113.35, 643.26, 1459.73, 640.79, 1459.73, 5...",58305.0,"[1113.35, 390.9, 346.38, 252.36]",0,{'occluded': False}
5418,5419,5873,11,"[[0.0, 1395.36, 2464.0, 1385.51, 2464.0, 524.4...",2128869.0,"[0.0, 494.82, 2464.0, 900.54]",0,{'occluded': False}
5419,5420,5881,12,"[[598.73, 322.86, 601.2, 334.0, 648.21, 329.05...",1975.0,"[598.73, 288.23, 96.49, 66.8]",0,{'occluded': False}
5420,5421,5881,12,"[[376.06, 132.36, 359.98, 144.73, 319.15, 158....",10263.0,"[319.15, 132.36, 165.77, 89.06]",0,{'occluded': False}
